In [ ]:
#================================= Start of importing required packages and libraries =========================================#
from __future__ import print_function
%matplotlib inline
import numpy as np
import torch
from experiment_federated import *
import random
#================================== End of importing required packages and libraries ==========================================#

In [ ]:
import torch
import random
import torch.nn as nn
import subprocess

# ========== 智能 GPU 检测函数 ==========
def get_free_gpu(threshold_mb=1024*20):
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_free = [int(x) for x in result.strip().split('\n')]
        for idx, mem in enumerate(memory_free):
            if mem > threshold_mb:
                return idx
    except Exception as e:
        print("无法检测GPU状态，默认使用CPU:", e)
    return None

# ========== 设备选择 ==========
free_gpu = get_free_gpu()
if free_gpu is not None and torch.cuda.is_available():
    DEVICE = torch.device(f'cuda:{free_gpu}')
    GPU_NAME = torch.cuda.get_device_name(free_gpu)
    print(f"✅ 使用空闲GPU: cuda:{free_gpu} - {GPU_NAME}")
else:
    DEVICE = torch.device('cpu')
    GPU_NAME = "CPU"
    print("⚠️ 没有可用GPU，改为使用CPU")

# ========== 环境标识 ==========
IS_SERVER = "A100" in GPU_NAME
IS_LOCAL = "3060" in GPU_NAME or "RTX 3060" in GPU_NAME

# ========== 基本超参数配置 ==========
DATASET_NAME = "PATHMNIST"
MODEL_NAME = "CNNPATH"
DD_TYPE = 'NON_IID'
ALPHA = 1
NUM_PEERS = 100
FRAC_PEERS = 1
SEED = 7
random.seed(SEED)
CRITERION = nn.CrossEntropyLoss()
NUM_CLASSES = 9
CLASS_PER_PEER = 9
SAMPLES_PER_CLASS = 582
RATE_UNBALANCE = 1

LABELS_DICT = {
    'Adipose': 0,
    'Background': 1,
    'Debris': 2,
    'Lymphocytes': 3,
    'Mucus': 4,
    'Smooth_muscle': 5,
    'Normal_colon_mucosa': 6,
    'Cancer_stroma': 7,
    'Adenocarcinoma_epithelium': 8
}
SOURCE_CLASS = 3
TARGET_CLASS = 5

# ========== 环境自适应参数配置 ==========
if IS_SERVER:
    GLOBAL_ROUNDS = 200           # 增加全局轮数，让模型充分学习
    LOCAL_EPOCHS = 3              # 本地训练较充分，降低攻击干扰
    LOCAL_BS = 64                 # 中等 batch size，兼顾稳定与速度
    LOCAL_LR = 0.01              # 降低学习率，增强收敛稳定性
    LOCAL_MOMENTUM = 0.9          # 保持稳定惯性
    TEST_BATCH_SIZE = 1000
    USE_LR_SCHEDULER = True       # 使用学习率调度器控制收敛后期收缩

elif IS_LOCAL:
    GLOBAL_ROUNDS = 5
    LOCAL_EPOCHS = 1
    TEST_BATCH_SIZE = 256
    LOCAL_BS = 32
else:
    GLOBAL_ROUNDS = 50
    LOCAL_EPOCHS = 2
    TEST_BATCH_SIZE = 512
    LOCAL_BS = 32

LOCAL_LR = 0.01
LOCAL_MOMENTUM = 0.9

# ========== 输出当前配置 ==========
print(f"[环境] GPU: {GPU_NAME}")
print(f"[配置] DEVICE = {DEVICE}, GLOBAL_ROUNDS = {GLOBAL_ROUNDS}, LOCAL_EPOCHS = {LOCAL_EPOCHS}, TEST_BATCH_SIZE = {TEST_BATCH_SIZE}")


In [ ]:
RULE = 'lfighter_dbo'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
# Baseline|: FedAvg-no attacks (FL)
RULE = 'fedavg'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0, 0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'lfighter'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'median'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'tmean'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'mkrum'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'foolsgold'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'Tolpegin'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'FLAME'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)